In [ ]:
!pip install xgboost
!pip install lightgbm

In [1]:
%load_ext autoreload
%autoreload 1
%aimport transformations

%aimport transformations
%aimport f

transformations loaded, version: 0.1
framework loaded, version: 0.1


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython.display import clear_output

import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 999
pd.options.display.max_rows = 113

import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.preprocessing import StandardScaler, PowerTransformer

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture

RS = 35577 # global random state seed
data_path = './data-raw/'

In [3]:
(X, y) = f.from_pkl(f'data-processed/all_b1_b2.data.pkl')

X.shape
X.columns

y.shape
y.value_counts()

(200, 36)

,x__amin,x__amax,x__sum,x__median,x__mean,x__std,x__var,x__q1,x__q3,x__iqr,x__kurtosis,x__skew,y__amin,y__amax,y__sum,y__median,y__mean,y__std,y__var,y__q1,y__q3,y__iqr,y__kurtosis,y__skew,z__amin,z__amax,z__sum,z__median,z__mean,z__std,z__var,z__q1,z__q3,z__iqr,z__kurtosis,z__skew
0,-6.716668,5.307868,-10988.070315,-0.079124,-0.134299,1.359987,1.849565,-0.944891,0.690446,1.635337,0.645580,-0.260035,-5.812267,5.638507,-2005.749440,-0.039563,-0.024515,1.090317,1.188792,-0.702004,0.622879,1.324883,0.845721,0.190106,-8.080924,8.244501,-15909.421691,-0.206307,-0.194449,1.503315,2.259956,-1.070594,0.754012,1.824606,0.873858,-0.226021
1,-12.103661,11.753020,-18844.323785,-0.175320,-0.239902,2.477716,6.139077,-1.522068,1.075231,2.597300,1.408079,-0.167536,-8.745936,12.452191,-318.869239,-0.134197,-0.004059,2.376120,5.645946,-1.459080,0.906782,2.365862,1.936212,0.887947,-10.193626,10.357203,-6543.554226,-0.110275,-0.083304,2.156482,4.650413,-1.262658,1.138140,2.400798,0.657898,0.065098
2,-7.678631,6.558420,-12546.141906,-0.079124,-0.136898,1.350739,1.824495,-0.752498,0.498054,1.250552,1.485456,-0.097227,-4.298115,5.259969,5780.428883,0.055072,0.063073,0.822131,0.675900,-0.323466,0.433610,0.757076,2.150199,0.258396,-10.289657,8.244501,12884.241287,0.177821,0.140587,1.480194,2.190974,-0.398371,0.850044,1.248415,3.049378,-0.710093


(200,)

1    100
0    100
Name: status, dtype: int64

# CV

In [4]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

In [37]:
def cross_val(model_factory, X, y, cv_repeat_n = 1, model_repeat_n = 1, folds_n = 7):
    np.random.seed(RS)
    get_random = lambda  : np.random.randint(1, 10000)
    cv_scores = []
    
    for i_cv in range(cv_repeat_n):
        fold_scores = []
        cv_scores.append(fold_scores)
        
        for i_fold, (idx_train, idx_test) in enumerate(StratifiedKFold(n_splits=folds_n, shuffle=True, random_state=get_random()).split(X, y, y)):
            
            X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
            X_test, y_test = X.iloc[idx_test], y.iloc[idx_test]
            
            model_scores = []
            fold_scores.append(model_scores)

            for i_model in range(model_repeat_n):
                rand = get_random()
                constructor, params_dic = model_factory(rand)
                
                model = constructor(**params_dic)
                # model = model_factory(get_random())
                model.fit(X_train, y_train)

                y_pred = model.predict_proba(X_test)[:, 1]
                score = roc_auc_score(y_test, y_pred)
                model_scores.append(score)
                
            model_name = model.__class__.__name__
            model_params = params_dic.copy()
                
    return model_name, model_params, cv_repeat_n, folds_n, model_repeat_n, np.array(cv_scores)

In [38]:
def get_stats_df(cv_results):
    
    ret_list = []
    
    for result in cv_results:
        
        model_params = result[1]
        model_params.pop('random_state', None)
        stats_ = result[-1]
        
        ret_list.append(
            {
                'model' : result[0],
                'params' : str(model_params).strip('{').strip('}'),
                'cv_repeat_n' : result[2],
                'folds_n' : result[3],    
                'model_repeat_n' : result[4],

                'min': stats_.min(),
                'mean': stats_.mean(),
                'max': stats_.max(),
                'std': stats_.std(),

                'folds_mean_min': stats_.mean(axis=2).reshape(1,-1).min(),
                # 'folds_mean_mean': stats_.mean(axis=2).reshape(1,-1).mean(),
                'folds_mean_max': stats_.mean(axis=2).reshape(1,-1).max(),
                'folds_mean_std': stats_.mean(axis=2).reshape(1,-1).std(),
            })
    return pd.DataFrame(ret_list)

def display_stats(df_stats):
    display(df_stats.style.format('{:,.2f}', df_stats.columns[5:])\
        .background_gradient(cmap='Blues', subset=df_stats.columns[5:9], axis=0)\
        .background_gradient(cmap='Greens', subset=df_stats.columns[9:], axis=0))

## Multiple Models

https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

In [158]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

# from sklearn.svm import SVC
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF
# from sklearn.naive_bayes import GaussianNB
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [159]:
import itertools

def grid_exec(func, *args):
    return [func(*element) for element in itertools.product(*args)]

def grid_exec_callback(func, callback, *args):
    return [ callback(func(*element)) for element in itertools.product(*args)]

In [165]:
all_models = [
    # lambda random_state: (DecisionTreeClassifier, {}),
    # lambda random_state: (KNeighborsClassifier,   {}),
    
    lambda random_state: (RandomForestClassifier, {'random_state':random_state, 'n_estimators': 10}),
    lambda random_state: (RandomForestClassifier, {'random_state':random_state, 'n_estimators': 25}),
    lambda random_state: (RandomForestClassifier, {'random_state':random_state, 'n_estimators': 75}),
    lambda random_state: (RandomForestClassifier, {'random_state':random_state, 'n_estimators': 150}),
    lambda random_state: (RandomForestClassifier, {'random_state':random_state, 'n_estimators': 300}),
    lambda random_state: (RandomForestClassifier, {'random_state':random_state, 'n_estimators': 600}),
    
    lambda random_state: (AdaBoostClassifier, {'random_state':random_state, 'n_estimators': 10}),
    lambda random_state: (AdaBoostClassifier, {'random_state':random_state, 'n_estimators': 25}),
    lambda random_state: (AdaBoostClassifier, {'random_state':random_state, 'n_estimators': 75}),
    lambda random_state: (AdaBoostClassifier, {'random_state':random_state, 'n_estimators': 150}),
    lambda random_state: (AdaBoostClassifier, {'random_state':random_state, 'n_estimators': 300}),
    lambda random_state: (AdaBoostClassifier, {'random_state':random_state, 'n_estimators': 600}),
    
    lambda random_state: (LGBMClassifier, {'random_state':random_state, 'n_estimators': 10}),
    lambda random_state: (LGBMClassifier, {'random_state':random_state, 'n_estimators': 25}),
    lambda random_state: (LGBMClassifier, {'random_state':random_state, 'n_estimators': 75}),
    lambda random_state: (LGBMClassifier, {'random_state':random_state, 'n_estimators': 150}),
    lambda random_state: (LGBMClassifier, {'random_state':random_state, 'n_estimators': 300}),
    lambda random_state: (LGBMClassifier, {'random_state':random_state, 'n_estimators': 600}),

    # lambda random_state: (XGBClassifier,          {'random_state':random_state}),
]

grid_func = lambda factory, cv_repeat_n, model_repeat_n, folds_n: cross_val(factory, X, y, cv_repeat_n, model_repeat_n, folds_n)

all_results = []
def on_1_completed(results):
    all_results.append(results)
    clear_output(wait=True)
    get_stats_df(all_results)
    display_stats(get_stats_df(all_results))

In [166]:
cv_repeat_list = [5]
model_repeat_list = [10]
folds_list = [5]

_ = grid_exec_callback(grid_func, on_1_completed, all_models, cv_repeat_list, model_repeat_list, folds_list)

,model,params,cv_repeat_n,folds_n,model_repeat_n,min,mean,max,std,folds_mean_min,folds_mean_max,folds_mean_std
0,RandomForestClassifier,'n_estimators': 10,5,5,10,0.91,0.98,1.00,0.02,0.92,1.00,0.02
1,RandomForestClassifier,'n_estimators': 25,5,5,10,0.92,0.98,1.00,0.02,0.94,1.00,0.02
2,RandomForestClassifier,'n_estimators': 75,5,5,10,0.93,0.99,1.00,0.02,0.94,1.00,0.02
3,RandomForestClassifier,'n_estimators': 150,5,5,10,0.94,0.99,1.00,0.02,0.95,1.00,0.02
4,RandomForestClassifier,'n_estimators': 300,5,5,10,0.94,0.99,1.00,0.02,0.95,1.00,0.02
5,RandomForestClassifier,'n_estimators': 600,5,5,10,0.94,0.99,1.00,0.02,0.95,1.00,0.02
6,AdaBoostClassifier,'n_estimators': 10,5,5,10,0.83,0.96,1.00,0.04,0.83,1.00,0.04
7,AdaBoostClassifier,'n_estimators': 25,5,5,10,0.84,0.97,1.00,0.04,0.84,1.00,0.04
8,AdaBoostClassifier,'n_estimators': 75,5,5,10,0.94,0.98,1.00,0.02,0.94,1.00,0.02
9,AdaBoostClassifier,'n_estimators': 150,5,5,10,0.94,0.99,1.00,0.02,0.95,1.00,0.02


In [157]:
# Rank

In [ ]:
for head in range(1, len(df_all_stats)):
    highest_fold_mean = df_all_stats.sort_values(by=['mean'], ascending = False).head(head).index.to_list()
    highest_min = df_all_stats.sort_values(by=['min'], ascending = False).head(head).index.to_list()
    lowest_folds_std_min = df_all_stats.sort_values(by=['folds_mean_std'], ascending = True).head(head).index.to_list()

    union = set(highest_fold_mean) & set(highest_min) & set(lowest_folds_std_min)
    
    if len(union) > 0:
        print(f'{head} - {union}')

In [162]:
df = df_all_stats.copy()

In [163]:
df['rank'] = (df['mean'].rank(ascending = False) + df['min'].rank(ascending = False) + df['folds_mean_std'].rank(ascending = True)).rank()
df.sort_values(by=['rank'])

,model,params,cv_repeat_n,folds_n,model_repeat_n,min,mean,max,std,folds_mean_min,folds_mean_max,folds_mean_std,rank
2,RandomForestClassifier,'n_estimators': 60,10,5,10,0.93250,0.985517,1.0,0.014227,0.944875,1.0000,0.013312,1.0
8,LGBMClassifier,'n_estimators': 60,10,5,10,0.94500,0.984250,1.0,0.015742,0.945000,1.0000,0.015742,2.0
5,AdaBoostClassifier,'n_estimators': 60,10,5,10,0.94250,0.982745,1.0,0.016601,0.942500,1.0000,0.016372,3.0
1,RandomForestClassifier,'n_estimators': 15,10,5,10,0.90875,0.980712,1.0,0.017469,0.936125,1.0000,0.014381,4.0
7,LGBMClassifier,'n_estimators': 15,10,5,10,0.90750,0.975200,1.0,0.020893,0.907500,1.0000,0.020893,5.0
0,RandomForestClassifier,'n_estimators': 5,10,5,10,0.83000,0.963767,1.0,0.027596,0.900375,0.9955,0.020297,6.0
6,LGBMClassifier,'n_estimators': 5,10,5,10,0.87625,0.958925,1.0,0.029925,0.876250,1.0000,0.029925,7.0
3,AdaBoostClassifier,'n_estimators': 5,10,5,10,0.83125,0.951475,1.0,0.034055,0.831250,1.0000,0.034055,8.5
4,AdaBoostClassifier,'n_estimators': 15,10,5,10,0.81125,0.966110,1.0,0.034060,0.811250,1.0000,0.034057,8.5
